# Лаб-4. Рекомендации для коротких сессий

In [64]:
import torch
from torch import nn

import pandas as pd
import numpy as np
import datetime

In [65]:
IS_CUDA_USED = False
device = "cuda" if torch.cuda.is_available() and IS_CUDA_USED else "cpu"
print(f'Device: {device}')

Device: cpu


В качестве датасета будем использовать архив из мудла.

Это немного предобработанная версия [eCommerce events history](https://www.kaggle.com/datasets/mkechinov/ecommerce-events-history-in-electronics-store)

In [66]:
# Как и в предыдущей лабораторной пишем собственный загрузчик датасета
class ECommerceDataset:
    def __init__(self, path):
        self.train_data = pd.read_csv(rf"{path}/train_data.csv")
        self.test_data = pd.read_csv(rf"{path}/test_data.csv")

        # Добавляем колонку с идентификаторами товаров (для эмбедингов)
        all_data = pd.concat([self.train_data, self.test_data])
        unique_items = all_data['product_id'].unique()
        item_to_idx = pd.Series(data=np.arange(len(unique_items)), index=unique_items)
        item_map = pd.DataFrame({'product_id': unique_items, 'product_index': item_to_idx[unique_items].values})
        self.train_data = pd.merge(self.train_data, item_map, on='product_id', how='inner')
        self.test_data  = pd.merge(self.test_data,  item_map, on='product_id', how='inner')

        # Сортируем датасет так, чтобы все сессии оказались рядом, а клики внутри сессии упорядочились по времени
        self.train_data.sort_values(['user_session', 'event_time'], inplace=True)
        self.test_data.sort_values(['user_session', 'event_time'], inplace=True)

# Загрузка большого датасета может занять некоторое время
dataset = ECommerceDataset('./eCommerce')

In [67]:
dataset.train_data

,event_time,product_id,user_session,product_index
32678,1604329884,80548,003pEktS1X,4865
34407,1607580196,630753,00ImhDtWxv,4292
21963,1607165660,387956,00xjwy5Rb6,8
31665,1607168978,387956,00xjwy5Rb6,8
23220,1611391773,738,00zEpCxZUK,1478
...,...,...,...,...
14766,1613148682,93765,zzaAzAFcYL,3193
15086,1613148695,93765,zzaAzAFcYL,3193
32226,1613408761,564777,zzveLpjyyb,1226
25067,1613409009,564777,zzveLpjyyb,1226


In [68]:
print(
    'Количество уникальных товаров',
    pd.concat([dataset.train_data, dataset.test_data])['product_id'].nunique(),
    '=',
    pd.concat([dataset.train_data, dataset.test_data])['product_index'].max() + 1
)

Количество уникальных товаров 15316 = 15316


За основу мы возьмём модель GRU4Rec из статьи

https://arxiv.org/pdf/1511.06939

Для обучения сети последовательными действиями пользователей, необходимо сконструировать минибатчи особым образом

![Формирование минибатчей](images/mini-batch-creation.png)

Так чтобы в инпуте батча содержался набор кликов пользователя, а в таргете набор следующих кликов из той же сессии

In [69]:
class ECommerceLoader():
    def __init__(self, data, batch_size, shuffle=False):
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.session_count = data['user_session'].nunique()

        # Делаем массив с индексами начала и конца каждой сессии
        session_sizes = np.array(data.groupby('user_session').size().cumsum())
        self.offsets = np.append([0], session_sizes)

    def __iter__(self):
        session_order = np.arange(self.session_count)
        if self.shuffle:
            np.random.shuffle(session_order)

        # Заводим список активных сессий, размером с батч
        active_sessions = np.arange(self.batch_size)
        next_session = self.batch_size # индекс следующей сессии
        start = self.offsets[session_order[active_sessions]]   # индексы начал активных сессий
        end = self.offsets[session_order[active_sessions] + 1] # индексы концов активных сессий

        closed_mask = list(active_sessions) # список сессий, которые открываются на текущей итерации
    
        while True:
            min_len = (end - start).min() # Количество итераций, которые мы можем пройти, пока не закончится какая-то сессия
            idx_target = self.data['product_index'].values[start]

            # Итерируем по сессиям до тех пор, пока какая-то не закончится
            for i in range(min_len - 1):
                idx_input = idx_target
                idx_target = self.data['product_index'].values[start + i + 1]
                input = torch.LongTensor(idx_input)
                target = torch.LongTensor(idx_target)
                yield input, target, closed_mask # маску мы будем использовать чтобы обнулять новые сессии
                closed_mask = []

            start = start + (min_len - 1)

            # Пробегаемся по сессиям, которые должны быть завершены
            closed_mask = np.arange(len(active_sessions))[(end - start) <= 1]
            for idx in closed_mask:
                # Если новых сессий нет, просто завершаемся
                if next_session >= len(self.offsets) - 1:
                    return
                # Обновляем значения для новой сессии
                active_sessions[idx] = next_session
                start[idx] = self.offsets[session_order[next_session]]
                end[idx]   = self.offsets[session_order[next_session] + 1]
                next_session += 1

batch_size = 10

train_loader = ECommerceLoader(dataset.train_data, batch_size, shuffle=True)
test_loader  = ECommerceLoader(dataset.test_data, batch_size)

Сама модель основана на рекуррентной архитектуры, такие сети помимо выходного состояния возвращают ещё и скрытое состояние, которое передаётся в сеть на следующей итерации, позволяя таким образом обрабатывать связанные последовательности данных (такие как текст или действия пользователя).

![рекуррентная сеть](images/Recurrent_neural_network_unfold.svg.png)

В данном случае используется архитектура GRU (это как LSTM, но ещё лучше)
![](images/lstm-gru.png)


In [70]:
class GRU4Rec(nn.Module):
    def __init__(self):
        super().__init__()

        embedding_size = 64
        self.hidden_size = 64
        item_size = 15316
        
        self.num_layers = 1
        self.state = torch.zeros([self.num_layers, batch_size, self.hidden_size])
        self.embedding = nn.Embedding(item_size, embedding_size)
        self.gru = nn.GRU(embedding_size, self.hidden_size, num_layers=self.num_layers, batch_first=True)
        self.output_layer = nn.Linear(self.hidden_size, item_size)
        self.dropout = nn.Dropout(0.5)

    # Перегрузка to чтобы состояние тоже перевести на девайс
    def to(self, device):
        self.state = self.state.to(device)
        return super().to(device)

    # Обнуляем состояние для новых сессий
    def update_state(self, mask=None):
        self.state.detach_()
        if mask is None:
            self.state = torch.zeros(
                self.num_layers, batch_size, self.hidden_size, device=device
            )
        else:
            self.state[:, mask, :] = 0

    def forward(self, input):
        self.update_state(mask=None)
        v = input.unsqueeze(1)
        v = self.embedding(v)
        v, self.state = self.gru(v, self.state) # (batch_size, 1, hidden_size)
        hidden = v.squeeze(1) # (batch_size, hidden_size)
        v = self.dropout(hidden)
        v = self.output_layer(v)
        return v

In [71]:
# Тренировка происходит и тестирование

def train_iteration(model, data_loader, loss_function, optimizer):
    model.train()

    for batch, (x, y, m) in enumerate(data_loader):
        x, y = x.to(device), y.to(device)
        # Не забываем обнулить состояние
        model.update_state(m)

        pred = model(x)
        loss = loss_function(pred, y)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}]")

def test(model, data_loader, loss_function):
    model.eval()

    loss, correct, count = 0, 0 ,0
    with torch.no_grad():
        for x, y, m in data_loader:
            count += 1
            x, y = x.to(device), y.to(device)
            model.update_state(m)
            pred = model(x)
            loss += loss_function(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    loss = loss / count
    correct /= count * batch_size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    pass


def train(epochs, model, loss_function, optimizer):
    for t in range(epochs):
        print(f"== Epoch {t + 1} ==")
        train_iteration(model, train_loader, loss_function, optimizer)
        test(model, test_loader, loss_function)

In [72]:
LEARNING_RATE = 0.001

model = GRU4Rec().to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

train(10, model, loss, optimizer)

== Epoch 1 ==
loss: 9.649190  [   10]
loss: 8.067137  [10010]
Test Error: 
 Accuracy: 2.7%, Avg loss: 10.411714 

== Epoch 2 ==
loss: 7.837283  [   10]
loss: 4.936797  [10010]
Test Error: 
 Accuracy: 3.3%, Avg loss: 11.482718 

== Epoch 3 ==
loss: 4.578692  [   10]
loss: 5.623654  [10010]
Test Error: 
 Accuracy: 3.6%, Avg loss: 12.242878 

== Epoch 4 ==
loss: 2.898281  [   10]
loss: 1.515950  [10010]
Test Error: 
 Accuracy: 3.7%, Avg loss: 12.657432 

== Epoch 5 ==
loss: 4.118772  [   10]
loss: 2.973674  [10010]
Test Error: 
 Accuracy: 3.8%, Avg loss: 12.899476 

== Epoch 6 ==
loss: 2.006286  [   10]
loss: 2.730523  [10010]
Test Error: 
 Accuracy: 3.8%, Avg loss: 13.271358 

== Epoch 7 ==
loss: 2.514724  [   10]
loss: 1.746758  [10010]
Test Error: 
 Accuracy: 3.9%, Avg loss: 13.787472 

== Epoch 8 ==
loss: 3.621876  [   10]
loss: 2.073610  [10010]
Test Error: 
 Accuracy: 3.9%, Avg loss: 14.359397 

== Epoch 9 ==
loss: 1.069592  [   10]
loss: 2.823932  [10010]
Test Error: 
 Accuracy: 4.

## Задания

Основные:
- Достичь точности в 3.5% на этом датасете - 5 баллов
- На основе GRU4Rec построить модель для датасета из предыдущей лабораторной (Movielens) - 5 баллов

Дополнительные задания:
- Реализовать одну из функций потерь BPR или TOP1 (https://arxiv.org/pdf/1511.06939) - 5 баллов
- Реализовать вторую функцию потерь - 5 баллов


## Полезные ссылки

Полезные ссылки по рекомендательным системам, модели из лекции и не только

- Репозиторий с кучей информации по рекомендательным системам https://github.com/recommenders-team/recommenders
- Рекомендательные системы на основе свёрток https://arxiv.org/pdf/1809.07426
- Sequence-Aware Factorization Machines (машина факторизации для временных последовательностей) https://arxiv.org/pdf/1911.02752

